In [85]:
print('Cosmic Ray Analysis Code by Shawn - main function outline:', end = '\n 1 ')
print('Import data file', end = '\n 2 ')
print('Modify data - rescale, detrend, remove re-zeros, etc.', end = '\n 3 ')
print('Matched filter peak finder', end = '\n 4 ')
print('Initial analysis tools - spectrum analyzer, etc.', end = '\n 5 ')
print('Fitter and fit rejection', end = '\n 6 ')
print('Coincidence analysis', end = '\n 7 ')
print('Plot')

Cosmic Ray Analysis Code by Shawn - main function outline:
 1 Import data file
 2 Modify data - rescale, detrend, remove re-zeros, etc.
 3 Matched filter peak finder
 4 Initial analysis tools - spectrum analyzer, etc.
 5 Fitter and fit rejection
 6 Coincidence analysis
 7 Plot


In [86]:
print('Imports')
import matplotlib
matplotlib.use('TkAgg')
import pylab as pl
#from DFT import *
import numpy as np
import numpy.polynomial.polynomial as poly
from scipy import signal
from scipy import optimize
from scipy import math
from scipy.fftpack import rfft, irfft, fftfreq, ifft, fft
from scipy.signal import argrelextrema
from scipy.signal import find_peaks_cwt
import scipy.integrate as integrate
import matplotlib.colors as colors
import matplotlib.ticker as ticker
import shutil
import statistics
from statistics import median_low
from matplotlib import cm
from matplotlib import mlab
from math import exp, expm1
import sys
import bisect
import os
import pickle

Imports


In [87]:
print('Supporting functions')

def extract_raw_data(fn,colNames):
    '''
    Extract txt data from a LabVIEW output file fn and return
    a Python dictionary. colNames is a list of strings
    corresponding to each column in the txt file.

    d['filename'] returns the filename.
    d[column name] returns an array with the data from that column.
    '''
    f = open(fn)
    d = {}
    d['filename'] = fn
    llist = []
    for i in range(len(colNames)):
        llist.append([])
    for line in f:
        lineSplit = line.split()
        for i in range(len(colNames)):
            llist[i].append(float(lineSplit[i].replace(',','')))
    for i in range(len(colNames)):
        d[colNames[i]] = pl.array(llist[i])
    f.close()
    return d


###########################################################################


def extract_raw_ts(fn):
    '''
    Extract XY data from a LabVIEW output file fn and
    return a Python dictionary with entries x and y.

    d['filename'] returns the filename.
    '''
    colNames = ['y0','y1','y2','y3','y4','y5','y6','y7']
    #colNames = ['y0','y1']
    return extract_raw_data(fn,colNames)

def extract_raw_ts_3ch(fn):
    colNames = ['y0','y1', 'y2']
    return extract_raw_data(fn,colNames)

def extract_raw_ts_1ch(fn):
    colNames = ['y0']
    return extract_raw_data(fn,colNames)


###########################################################################


def template(x):
    xsc = 100.
    relsc = 100./7.
    phs = 0.4975
    if(x<0.5): 
        return 1/(1+math.exp(-xsc*relsc*(x-phs)))
    else:
        return math.exp(-xsc*(x-0.499848))

def filtertempl(x):
    return 1 - math.exp(-(10.*x))


###########################################################################


def CR_combine(fnList,newfn):
    with open(newfn + '.dat','wb') as wfd:
        for f in fnList:
            with open(f,'rb') as fd:
                shutil.copyfileobj(fd, wfd, 1024*1024*10)


###########################################################################


def find_nearest(array, value):
    array = np.asarray(array)
    minarr = array - value
    idx = np.argmin(i for i in minarr if i > 0)
    idxx = (np.abs(array - value)).argmin()

    return array[idxx].tolist()

def find_nearest_fix(array, value):
    array = np.asarray(array)
    minarr = array - value
    idx = np.argmin(i for i in minarr if i > 0)
    idxx = (np.abs(array - value)).argmin()

    return array[idx].tolist()



Supporting functions


In [88]:


with open ('/Users/shawn/Google Drive/My Drive/Python/Cosmic Rays/2021_05_17-control_source/CR_Scan_002/energies_1-500', 'rb') as fp:
    energies_cs = pickle.load(fp)
with open ('/Users/shawn/Google Drive/My Drive/Python/Cosmic Rays/2021_05_17-control_source/CR_Scan_002/energies_data_1-500', 'rb') as fp:
    energies_data_cs = pickle.load(fp)
with open ('/Users/shawn/Google Drive/My Drive/Python/Cosmic Rays/2021_05_17-control_source/CR_Scan_002/timeconsts_1-500', 'rb') as fp:
    timeconsts_cs = pickle.load(fp)
with open ('/Users/shawn/Google Drive/My Drive/Python/Cosmic Rays/2021_05_17-control_source/CR_Scan_002/ratesall_1-500', 'rb') as fp:
    ratesall_cs = pickle.load(fp)
with open ('/Users/shawn/Google Drive/My Drive/Python/Cosmic Rays/2021_05_17-control_source/CR_Scan_002/allcoinc_1-500', 'rb') as fp:
    allcoinc_cs = pickle.load(fp)
with open ('/Users/shawn/Google Drive/My Drive/Python/Cosmic Rays/2021_05_17-control_source/CR_Scan_002/jfe_datasets_1-500', 'rb') as fp:
    jfe_datasets_cs = pickle.load(fp)
with open ('/Users/shawn/Google Drive/My Drive/Python/Cosmic Rays/2021_05_17-control_source/CR_Scan_002/numerrors_1-500', 'rb') as fp:
    numerrors_cs = pickle.load(fp)

    
    
with open ('/Users/shawn/Google Drive/My Drive/Python/Cosmic Rays/2021_05_28-mitig_source/CR_Scan_003/energies_1-500', 'rb') as fp:
    energies_ms = pickle.load(fp)
with open ('/Users/shawn/Google Drive/My Drive/Python/Cosmic Rays/2021_05_28-mitig_source/CR_Scan_003/energies_data_1-500', 'rb') as fp:
    energies_data_ms  = pickle.load(fp)
with open ('/Users/shawn/Google Drive/My Drive/Python/Cosmic Rays/2021_05_28-mitig_source/CR_Scan_003/timeconsts_1-500', 'rb') as fp:
    timeconsts_ms  = pickle.load(fp)
with open ('/Users/shawn/Google Drive/My Drive/Python/Cosmic Rays/2021_05_28-mitig_source/CR_Scan_003/ratesall_1-500', 'rb') as fp:
    ratesall_ms  = pickle.load(fp)
with open ('/Users/shawn/Google Drive/My Drive/Python/Cosmic Rays/2021_05_28-mitig_source/CR_Scan_003/allcoinc_1-500', 'rb') as fp:
    allcoinc_ms  = pickle.load(fp)
with open ('/Users/shawn/Google Drive/My Drive/Python/Cosmic Rays/2021_05_28-mitig_source/CR_Scan_003/jfe_datasets_1-500', 'rb') as fp:
    jfe_datasets_ms  = pickle.load(fp)
with open ('/Users/shawn/Google Drive/My Drive/Python/Cosmic Rays/2021_05_28-mitig_source/CR_Scan_003/numerrors_1-500', 'rb') as fp:
    numerrors_ms  = pickle.load(fp)




In [89]:
transp = 0.75

#num_bins1 = np.logspace(np.log10(0.1),np.log10(100),bm*25)
#num_bins1 = bm*25

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 



pl.clf() 
n1, bins1, patchs1 = pl.hist(energies_cs[0], bins='auto', alpha = transp, log=True,  label = 'bolo 1')
n2, bins2, patchs2 = pl.hist(energies_cs[1], bins='auto', alpha = transp, log=True, label = 'bolo 2')
n3, bins3, patchs3 = pl.hist(energies_cs[2], bins='auto', alpha = transp, log=True, label = 'bolo 3')
#pl.xscale('log')
#pl.xlim(xmin = 0, xmax = np.max(np.max(energies_cs[0]+energies_cs[1]+energies_cs[2])))
pl.xlim(xmin = 0., xmax =200.)
pl.title("Energy from pulse fits of non-mitigated bolometers, Co-60")
pl.xlabel("Pulse energy (arb. units)")
pl.legend(loc='upper right')
pl.show()


pl.clf()
n2, bins2, patchs2 = pl.hist(energies_ms[1], bins='auto', alpha = transp, log=True, label = 'bolo 2')
n1, bins1, patchs1 = pl.hist(energies_ms[0], bins='auto', alpha = transp, log=True,  label = 'bolo 1')

n3, bins3, patchs3 = pl.hist(energies_ms[2], bins='auto', alpha = transp, log=True, label = 'bolo 3')
#pl.xscale('log')
#pl.xlim(xmin = 0, xmax = np.max(np.max(energies_ms[0]+energies_ms[1]+energies_ms[2])))
pl.xlim(xmin = 0., xmax =125.)
pl.title("Energy from pulse fits of mitigated bolometers, Co-60")
pl.xlabel("Pulse energy (arb. units)")
pl.legend(loc='upper right')
pl.show()



# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 



pl.clf() 
n1, bins1, patchs1 = pl.hist(energies_data_cs[0], bins='auto', alpha = transp, log=True,  label = 'bolo 1')
n2, bins2, patchs2 = pl.hist(energies_data_cs[1], bins='auto', alpha = transp, log=True, label = 'bolo 2')
n3, bins3, patchs3 = pl.hist(energies_data_cs[2], bins='auto', alpha = transp, log=True, label = 'bolo 3')
#pl.xscale('log')
#pl.xlim(xmin = 0, xmax = np.max(np.max(energies_data_cs[0]+energies_data_cs[1]+energies_data_cs[2])))
pl.xlim(xmin = 0., xmax =1.)
pl.title("Energy from pulse data of non-mitigated bolometers, Co-60")
pl.xlabel("Pulse energy (arb. units)")
pl.legend(loc='upper right')
pl.show()


pl.clf()
n3, bins3, patchs3 = pl.hist(energies_data_ms[2], bins='auto', alpha = transp, log=True, label = 'bolo 3')

n2, bins2, patchs2 = pl.hist(energies_data_ms[1], bins='auto', alpha = transp, log=True, label = 'bolo 2')
n1, bins1, patchs1 = pl.hist(energies_data_ms[0], bins='auto', alpha = transp, log=True,  label = 'bolo 1')

#pl.xscale('log')
#pl.xlim(xmin = 0, xmax = np.max(np.max(energies_data_ms[0]+energies_data_ms[1]+energies_data_ms[2])))
pl.xlim(xmin = 0., xmax =0.25)
pl.title("Energy from pulse data of mitigated bolometers, Co-60")
pl.xlabel("Pulse energy (arb. units)")
pl.legend(loc='upper right')
pl.show()



# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 


pl.clf() 
n1, bins1, patchs1 = pl.hist(timeconsts_cs[0], bins='auto', alpha = transp, log=True,  label = 'bolo 1')
n2, bins2, patchs2 = pl.hist(timeconsts_cs[1], bins='auto', alpha = transp, log=True, label = 'bolo 2')
n3, bins3, patchs3 = pl.hist(timeconsts_cs[2], bins='auto', alpha = transp, log=True, label = 'bolo 3')
#pl.xscale('log')
#pl.xlim(xmin = 0, xmax = np.max(np.max(timeconsts_cs[0]+timeconsts_cs[1]+timeconsts_cs[2])))
pl.xlim(xmin = 0., xmax =0.015)
pl.title("Time constants of non-mitigated bolometers, Co-60")
pl.xlabel("Time constants (ms)")
pl.legend(loc='upper right')
pl.show()


pl.clf() 
n1, bins1, patchs1 = pl.hist(timeconsts_ms[0], bins='auto', alpha = transp, log=True,  label = 'bolo 1')
n2, bins2, patchs2 = pl.hist(timeconsts_ms[1], bins='auto', alpha = transp, log=True, label = 'bolo 2')
n3, bins3, patchs3 = pl.hist(timeconsts_ms[2], bins='auto', alpha = transp, log=True, label = 'bolo 3')
#pl.xscale('log')
pl.xlim(xmin = 0, xmax = np.max(np.max(timeconsts_ms[0]+timeconsts_ms[1]+timeconsts_ms[2])))
#pl.xlim(xmin = 0., xmax =200.)
pl.title("Time constants of mitigated bolometers, Co-60")
pl.xlabel("Time constants (ms)")
pl.legend(loc='upper right')
pl.show()


In [91]:
print(len(allcoinc_cs[0])+len(allcoinc_cs[2])+len(allcoinc_cs[4]))
print(len(allcoinc_ms[0])+len(allcoinc_ms[2])+len(allcoinc_ms[4]))

print(np.mean(timeconsts_cs[0]), np.mean(timeconsts_cs[1]), np.mean(timeconsts_cs[2]))
print(np.mean(timeconsts_ms[0]), np.mean(timeconsts_ms[1]), np.mean(timeconsts_ms[2]))
print(len(energies_cs[0]))
print(len(energies_ms[0]))

print(np.mean(energies_cs[0]), np.mean(energies_cs[1]), np.mean(energies_cs[2]))
print(np.mean(energies_ms[0]), np.mean(energies_ms[1]), np.mean(energies_ms[2]))


141
506
0.005969404186795041 0.007082910321488678 0.00632280701754375
0.008050381679390319 0.006810501750291759 0.004789576547231241
621
1179
30.812280754233104 32.05032394483804 6.4191695060341685
8.968287755034668 17.599357466166158 8.511047780321425
